In [2]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt


In [3]:
part_0 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00000-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_1 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00001-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_2 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00002-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_3 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00003-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_4 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00004-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_5 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00005-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_6 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00006-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_7 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00007-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_8 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00008-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
part_9 = pd.read_parquet('D:\Datasets\grab-posis-city=Singapore\part-00009-8bbff892-97d2-4011-9961-703e38972569.c000.snappy.parquet')
main_df = pd.concat([part_0, part_1, part_2, part_3, part_4, part_5, part_6, part_7, part_8, part_9])




In [4]:
main_df.head()

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy
0,70014,car,android,1554943236,1.342326,103.888969,18.910000,248,3.9
1,73573,car,android,1555582623,1.321781,103.856366,17.719076,44,4.0
2,75567,car,android,1555141026,1.327088,103.861273,14.021548,34,3.9
3,1410,car,android,1555731693,1.262482,103.823794,13.026521,181,4.0
4,4354,car,android,1555584497,1.283799,103.807210,14.812943,93,3.9


In [5]:
len(main_df)

30329685

In [6]:
len(unique(main_df['trj_id']))

28000

In [7]:
total_trips = unique(main_df['trj_id'])

In [8]:
total_trips

array(['10', '100', '1000', ..., '998', '9982', '9984'], dtype=object)

In [9]:
trips_train, trips_test = train_test_split(total_trips, test_size=0.15, random_state=16)

In [10]:
print(trips_train)
print(trips_test)
print(len(trips_train))
print(len(trips_test))

['76127' '42857' '82833' ... '21467' '64126' '37522']
['59490' '79051' '61595' ... '59686' '59197' '41361']
23800
4200


In [11]:
train_df = main_df.loc[main_df['trj_id'].isin(trips_train)]
test_df = main_df.loc[main_df['trj_id'].isin(trips_test)]

In [12]:
train_df.head()

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy
0,70014,car,android,1554943236,1.342326,103.888969,18.910000,248,3.9
1,73573,car,android,1555582623,1.321781,103.856366,17.719076,44,4.0
3,1410,car,android,1555731693,1.262482,103.823794,13.026521,181,4.0
4,4354,car,android,1555584497,1.283799,103.807210,14.812943,93,3.9
5,32630,car,android,1555395258,1.300330,103.906175,23.238184,73,3.9


In [44]:
train_df['geohash'] = train_df.apply(lambda x: gh.encode(x.rawlat, x.rawlng, precision=7),axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
train_df['datetime'] = pd.to_datetime(train_df['pingtimestamp'],unit='s')
train_df['hour'] = train_df['datetime'].dt.hour
train_df['day'] = train_df['datetime'].dt.dayofweek

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [14]:
train_df.head(20)

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,datetime,hour,day
0,70014,car,android,1554943236,1.342326,103.888969,18.910000,248,3.900,2019-04-11 00:40:36,0,3
1,73573,car,android,1555582623,1.321781,103.856366,17.719076,44,4.000,2019-04-18 10:17:03,10,3
3,1410,car,android,1555731693,1.262482,103.823794,13.026521,181,4.000,2019-04-20 03:41:33,3,5
4,4354,car,android,1555584497,1.283799,103.807210,14.812943,93,3.900,2019-04-18 10:48:17,10,3
5,32630,car,android,1555395258,1.300330,103.906175,23.238184,73,3.900,2019-04-16 06:14:18,6,1
6,64631,car,android,1554768955,1.351751,103.759809,0.000000,82,3.000,2019-04-09 00:15:55,0,1
7,76484,car,android,1554783532,1.339442,103.886003,3.975536,321,3.649,2019-04-09 04:18:52,4,1
8,65747,car,ios,1554898418,1.291847,103.842375,4.681444,324,50.000,2019-04-10 12:13:38,12,2
9,60478,car,ios,1555593189,1.330900,103.815431,16.530499,31,16.000,2019-04-18 13:13:09,13,3
10,46987,car,ios,1555853886,1.442025,103.811432,20.959999,203,10.000,2019-04-21 13:38:06,13,6


In [47]:
train_df = train_df.sort_values(['trj_id','pingtimestamp'])

In [48]:
train_df.head(20)

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,geohash,datetime,hour,day
2848570,100,car,ios,1555382655,1.345079,103.938474,2.549204,181,11.0,w21zthv,2019-04-16 02:44:15,2,1
1237719,100,car,ios,1555382656,1.345010,103.938480,2.549204,181,11.0,w21zthv,2019-04-16 02:44:16,2,1
841281,100,car,ios,1555382657,1.344927,103.938480,3.934038,182,11.0,w21zthv,2019-04-16 02:44:17,2,1
839196,100,car,ios,1555382658,1.344850,103.938480,9.536406,183,11.0,w21zthv,2019-04-16 02:44:18,2,1
1280718,100,car,ios,1555382659,1.344713,103.938477,10.293652,185,11.0,w21zthv,2019-04-16 02:44:19,2,1
2502672,100,car,ios,1555382660,1.344603,103.938473,10.293652,189,11.0,w21zthv,2019-04-16 02:44:20,2,1
1681350,100,car,ios,1555382661,1.344488,103.938464,10.976644,193,12.0,w21zthv,2019-04-16 02:44:21,2,1
2926753,100,car,ios,1555382662,1.344365,103.938442,12.921396,196,12.0,w21ztht,2019-04-16 02:44:22,2,1
1612506,100,car,ios,1555382663,1.344231,103.938409,13.301397,200,12.0,w21ztht,2019-04-16 02:44:23,2,1
268082,100,car,ios,1555382664,1.344114,103.938375,13.102533,202,13.0,w21ztht,2019-04-16 02:44:24,2,1


In [96]:
#test_train = train_df.head(100)
main_arr = []
initial_trj_id = ''
initial_geohash = ''
initial_timestamp = ''

for row in train_df.iterrows():
    # if current trip id is the same as the previous id, check for difference in geohash
    if row[1]['trj_id'] == initial_trj_id:
        if row[1]['geohash'] != initial_geohash:
            temp_arr = []
            temp_arr.append(row[1]['trj_id'])
            temp_arr.append(initial_geohash)
            temp_arr.append(row[1]['geohash'])
            temp_arr.append(initial_timestamp)
            temp_arr.append(row[1]['datetime'])
            #print(initial_timestamp, 'initial_timestamp')
            #print(row[1]['datetime'], 'row datetime')
            #print(row[1]['datetime'] - initial_timestamp, 'difference')
            temp_arr.append((row[1]['datetime'] -initial_timestamp).total_seconds())
            temp_arr.append(row[1]['hour'])
            temp_arr.append(row[1]['day'])
            
            initial_timestamp = row[1]['datetime']
            initial_geohash = row[1]['geohash']
            
            main_arr.append(temp_arr)
            
        
    elif row[1]['trj_id'] != initial_trj_id:
        initial_trj_id = row[1]['trj_id']
        initial_timestamp = row[1]['datetime']
        initial_geohash = row[1]['geohash']
        
    
    # if current trip id is different, start a new id
    #print(row[1]['geohash'])

In [97]:
view = pd.DataFrame(main_arr)
view.head(20)

,0,1,2,3,4,5,6,7
0,100,w21zthv,w21ztht,2019-04-16 02:44:15,2019-04-16 02:44:22,7.0,2,1
1,100,w21ztht,w21zthm,2019-04-16 02:44:22,2019-04-16 02:44:32,10.0,2,1
2,100,w21zthm,w21zthk,2019-04-16 02:44:32,2019-04-16 02:44:37,5.0,2,1
3,100,w21zthk,w21zthh,2019-04-16 02:44:37,2019-04-16 02:44:41,4.0,2,1
4,100,w21zthh,w21zt5u,2019-04-16 02:44:41,2019-04-16 02:44:50,9.0,2,1
5,100,w21zt5u,w21zt5g,2019-04-16 02:44:50,2019-04-16 02:44:53,3.0,2,1
6,100,w21zt5g,w21zt5e,2019-04-16 02:44:53,2019-04-16 02:45:04,11.0,2,1
7,100,w21zt5e,w21zt5d,2019-04-16 02:45:04,2019-04-16 02:45:11,7.0,2,1
8,100,w21zt5d,w21zt59,2019-04-16 02:45:11,2019-04-16 02:46:17,66.0,2,1
9,100,w21zt59,w21zt53,2019-04-16 02:46:17,2019-04-16 02:46:23,6.0,2,1


In [98]:
len(view)

3495658

In [101]:
view.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day
0,100,w21zthv,w21ztht,2019-04-16 02:44:15,2019-04-16 02:44:22,7.0,2,1
1,100,w21ztht,w21zthm,2019-04-16 02:44:22,2019-04-16 02:44:32,10.0,2,1
2,100,w21zthm,w21zthk,2019-04-16 02:44:32,2019-04-16 02:44:37,5.0,2,1
3,100,w21zthk,w21zthh,2019-04-16 02:44:37,2019-04-16 02:44:41,4.0,2,1
4,100,w21zthh,w21zt5u,2019-04-16 02:44:41,2019-04-16 02:44:50,9.0,2,1


In [100]:
view.columns = ['trj_id', 'geohash_source', 'geohash_dest', 'timestamp_source', 'timestamp_dest', 'time_taken', 'hour','day']
        
    

In [103]:
view.to_csv('processed_geohash.csv',index=False)

In [10]:
len(df)

3032569

In [69]:
test['geohash'] = test.apply(lambda x: gh.encode(x.rawlat, x.rawlng, precision=8),axis=1)
test['datetime'] = pd.to_datetime(test['pingtimestamp'],unit='s')
test['hour'] = test['datetime'].dt.hour

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [17]:
print(len(test_df))
test_df.head()

4538468


,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy
2,75567,car,android,1555141026,1.327088,103.861273,14.021548,34,3.900
33,78938,car,ios,1555031505,1.283816,103.836612,20.009304,221,4.000
36,82760,car,android,1555580996,1.355439,103.698673,2.818491,217,4.943
40,21150,car,android,1555129486,1.376716,103.858606,0.590000,72,12.103
52,45510,car,android,1555501661,1.371317,103.860372,25.019993,339,3.900


In [18]:
test_df['datetime'] = pd.to_datetime(test_df['pingtimestamp'],unit='s')
test_df['hour'] = test_df['datetime'].dt.hour
test_df['day'] = test_df['datetime'].dt.dayofweek

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [15]:
train_df = train_df.sort_values(by=['trj_id', 'datetime'])
train_df_agg = train_df.groupby('trj_id').agg(['first', 'last']).stack().reset_index()
train_df_agg.head()

,trj_id,level_1,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,datetime,hour,day
0,100,first,car,ios,1555382655,1.345079,103.938474,2.549204,181,11.0,2019-04-16 02:44:15,2,1
1,100,last,car,ios,1555383648,1.335207,103.842207,5.622902,208,13.0,2019-04-16 03:00:48,3,1
2,1000,first,car,ios,1554943773,1.435317,103.788645,0.327651,192,16.0,2019-04-11 00:49:33,0,3
3,1000,last,car,ios,1554944629,1.368858,103.861007,8.445251,180,8.0,2019-04-11 01:03:49,1,3
4,10004,first,car,ios,1554779556,1.433899,103.768705,20.158873,176,8.0,2019-04-09 03:12:36,3,1


In [17]:
train_main_arr = []
temp_arr = []
for row in train_df_agg.iterrows():
    
    if row[1]['level_1'] == 'first':
        temp_arr.append(row[1]['trj_id'])
        temp_arr.append(row[1]['rawlat'])
        temp_arr.append(row[1]['rawlng'])
        temp_arr.append(row[1]['datetime'])
        
    elif row[1]['level_1'] == 'last':
        temp_arr.append(row[1]['rawlat'])
        temp_arr.append(row[1]['rawlng'])
        temp_arr.append(row[1]['datetime'])
        train_main_arr.append(temp_arr)
        temp_arr = []
        
        
final_train = pd.DataFrame(train_main_arr)
final_train.columns = ['trj_id', 'source_lat', 'source_lng', 'source_timestamp', 'dest_lat', 'dest_lng', 'dest_timestamp']
final_train['time_taken'] = final_train['dest_timestamp'] - final_train['source_timestamp']
final_train['time_taken'] = final_train['time_taken'].astype('timedelta64[s]')
final_train['geohash_source'] = final_train.apply(lambda x: gh.encode(x.source_lat, x.source_lng, precision=7),axis=1)
final_train['geohash_dest'] = final_train.apply(lambda x: gh.encode(x.dest_lat, x.dest_lng, precision=7),axis=1)
final_train['hour'] = final_train['source_timestamp'].dt.hour
final_train['day'] = final_train['source_timestamp'].dt.dayofweek
final_train.head()

,trj_id,source_lat,source_lng,source_timestamp,dest_lat,dest_lng,dest_timestamp,time_taken,geohash_source,geohash_dest,hour,day
0,100,1.345079,103.938474,2019-04-16 02:44:15,1.335207,103.842207,2019-04-16 03:00:48,993.0,w21zthv,w21zdgp,2,1
1,1000,1.435317,103.788645,2019-04-11 00:49:33,1.368858,103.861007,2019-04-11 01:03:49,856.0,w23b1v2,w21zg3j,0,3
2,10004,1.433899,103.768705,2019-04-09 03:12:36,1.392301,103.908846,2019-04-09 03:26:47,851.0,w23b1m4,w21zut2,3,1
3,10005,1.371245,103.871188,2019-04-17 01:21:40,1.301454,103.843052,2019-04-17 01:38:48,1028.0,w21zg9s,w21z7hb,1,2
4,1001,1.344444,103.726811,2019-04-13 12:11:50,1.330114,103.875455,2019-04-13 12:30:11,1101.0,w21z8ke,w21zedp,12,5


In [18]:
final_train.to_csv('train_agg.csv', index=False)

In [19]:
test_df.head()

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,datetime,hour,day
2,75567,car,android,1555141026,1.327088,103.861273,14.021548,34,3.900,2019-04-13 07:37:06,7,5
33,78938,car,ios,1555031505,1.283816,103.836612,20.009304,221,4.000,2019-04-12 01:11:45,1,4
36,82760,car,android,1555580996,1.355439,103.698673,2.818491,217,4.943,2019-04-18 09:49:56,9,3
40,21150,car,android,1555129486,1.376716,103.858606,0.590000,72,12.103,2019-04-13 04:24:46,4,5
52,45510,car,android,1555501661,1.371317,103.860372,25.019993,339,3.900,2019-04-17 11:47:41,11,2


In [22]:
test_df = test_df.sort_values(by=['trj_id', 'datetime'])
test_df.head()

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,datetime,hour,day
1308294,10,car,android,1554809147,1.301775,103.799257,5.851406,133,8.000,2019-04-09 11:25:47,11,1
1304161,10,car,android,1554809148,1.301739,103.799290,4.786575,134,8.000,2019-04-09 11:25:48,11,1
2363198,10,car,android,1554809149,1.301719,103.799302,3.769620,132,6.967,2019-04-09 11:25:49,11,1
2745685,10,car,android,1554809150,1.301698,103.799316,3.317640,131,6.517,2019-04-09 11:25:50,11,1
2354803,10,car,android,1554809151,1.301676,103.799337,3.176355,130,5.425,2019-04-09 11:25:51,11,1


In [27]:
test_df = test_df.groupby('trj_id').agg(['first', 'last']).stack().reset_index()

In [28]:
test_df.head()

,trj_id,level_1,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,datetime,hour,day
0,10,first,car,android,1554809147,1.301775,103.799257,5.851406,133,8.000,2019-04-09 11:25:47,11,1
1,10,last,car,android,1554810296,1.358001,103.845160,0.000000,196,6.534,2019-04-09 11:44:56,11,1
2,10001,first,car,android,1555502175,1.375079,103.833412,-1.000000,0,3.000,2019-04-17 11:56:15,11,2
3,10001,last,car,android,1555503425,1.328267,103.750637,15.385287,303,3.900,2019-04-17 12:17:05,12,2
4,10010,first,car,android,1555822161,1.325336,103.726036,2.400166,131,3.424,2019-04-21 04:49:21,4,6


In [31]:
test_main_arr = []
temp_arr = []
for row in test_df.iterrows():
    
    if row[1]['level_1'] == 'first':
        temp_arr.append(row[1]['trj_id'])
        temp_arr.append(row[1]['rawlat'])
        temp_arr.append(row[1]['rawlng'])
        temp_arr.append(row[1]['datetime'])
        
    elif row[1]['level_1'] == 'last':
        temp_arr.append(row[1]['rawlat'])
        temp_arr.append(row[1]['rawlng'])
        temp_arr.append(row[1]['datetime'])
        test_main_arr.append(temp_arr)
        temp_arr = []

In [44]:
final_test = pd.DataFrame(test_main_arr)
final_test.columns = ['trj_id', 'source_lat', 'source_lng', 'source_timestamp', 'dest_lat', 'dest_lng', 'dest_timestamp']
final_test['time_taken'] = final_test['dest_timestamp'] - final_test['source_timestamp']
final_test['time_taken'] = final_test['time_taken'].astype('timedelta64[s]')
final_test['geohash_source'] = final_test.apply(lambda x: gh.encode(x.source_lat, x.source_lng, precision=7),axis=1)
final_test['geohash_dest'] = final_test.apply(lambda x: gh.encode(x.dest_lat, x.dest_lng, precision=7),axis=1)
final_test['hour'] = final_test['source_timestamp'].dt.hour
final_test['day'] = final_test['source_timestamp'].dt.dayofweek
final_test.head()

,trj_id,source_lat,source_lng,source_timestamp,dest_lat,dest_lng,dest_timestamp,time_taken,geohash_source,geohash_dest,hour,day
0,10,1.301775,103.799257,2019-04-09 11:25:47,1.358001,103.845160,2019-04-09 11:44:56,1149.0,w21z6hb,w21zep1,11,1
1,10001,1.375079,103.833412,2019-04-17 11:56:15,1.328267,103.750637,2019-04-17 12:17:05,1250.0,w21zff3,w21z8cu,11,2
2,10010,1.325336,103.726036,2019-04-21 04:49:21,1.408185,103.895839,2019-04-21 05:28:30,2349.0,w21z836,w23bh0q,4,6
3,10018,1.336971,103.706410,2019-04-17 03:12:03,1.288159,103.794293,2019-04-17 03:37:10,1507.0,w21xxgk,w21z3fs,3,2
4,10087,1.401270,103.880918,2019-04-08 09:55:14,1.288438,103.793752,2019-04-08 10:12:48,1054.0,w21zgz5,w21z3fs,9,0


In [45]:
final_test.to_csv("test_set.csv", index=False)

In [23]:
df = pd.DataFrame(np.arange(20).reshape(10, -1),
                  [['a', 'a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd'],
                   ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']],
                  ['X', 'Y'])

df.head()
   

X  Y
a a  0  1
  b  2  3
  c  4  5
  d  6  7
b e  8  9